In [4]:
## Revision 4
## This is a script to generate a comtrade file in ascii format with voltage and current modulations

## Goals 
## 1- All configs should be in a variable with key value pairs
## 2- Create modular functions which perform specific functions

import matplotlib.pyplot as plt
import math
import numpy as np
from scipy.fft import fft, fftfreq
from comtradeConfig import ComtradeConfig
from utilities import Utilities

    

Utilities.write_cfg_to_file()

def generate_signal(t,peak,base_frequency,phase_base, modulating_index,modulating_frequency, modulant_phase):
    signal = []
    for t_inst in t:
        signal.append(peak*math.cos( 2*math.pi*base_frequency*t_inst + (phase_base*(math.pi/180)) ) * ( 1+modulating_index_voltage*math.cos(2*math.pi*modulating_frequency*t_inst + (modulant_phase*(math.pi/180)) )))
    
    return signal

## generate data for dat file

fs = ComtradeConfig.sampling_rate()
T = 1/fs

t = np.arange(0,ComtradeConfig.seconds(),T)

modulating_index = 0
modulating_frequency = 30


##generate_signal(t,ComtradeConfig.voltage_peak(100.5),ComtradeConfig.frequency(),0,modulating_index,modulating_frequency,)

## variables to hold 3 voltages
samples_va = []
samples_vb = []
samples_vc = []

## variables to hold 3 currents
samples_ia = []
samples_ib = []
samples_ic = []

voltage_peak = ComtradeConfig.voltage_peak()
current_peak = ComtradeConfig.current_peak()
base_frequency = ComtradeConfig.frequency()

## define the phases of va, vb and vc
phase_of_base_va = 0
phase_of_base_vb = phase_of_base_va - 120
phase_of_base_vc = phase_of_base_va + 120

## phase difference between v(i) and i(i) where i represents the a,b or c
lag = -30  


## define the phases of ia,ib and ic
phase_of_base_ia = phase_of_base_va + lag
phase_of_base_ib = phase_of_base_vb + lag
phase_of_base_ic = phase_of_base_vc + lag

## define variables to hold voltage and current alpha and beta components
samples_v_alpha = []
samples_v_beta  = []
samples_v_gamma = []

samples_i_alpha = []
samples_i_beta  = []
samples_i_gamma = []

##  define variables to hold P_park and Q_park
samples_P_park = []
samples_Q_park = []

## Defining Modulation index 
modulating_index_voltage = 0.2
modulating_index_current = 0

## modulating Frequency
modulating_frequency = 45
phase_of_modulating_frequency = 0


phase_of_voltage_modulant = 0
phase_of_current_modulant = 180

three_phase_frequency_shift = 120


for t_inst in t:
    # Generate values for all voltages
    va_inst = voltage_peak*math.cos( 2*math.pi*base_frequency*t_inst + (phase_of_base_va*(math.pi/180)) ) * ( 1+modulating_index_voltage*math.cos(2*math.pi*modulating_frequency*t_inst + (phase_of_voltage_modulant*(math.pi/180)) ))
    vb_inst = voltage_peak*math.cos( 2*math.pi*base_frequency*t_inst + (phase_of_base_vb*(math.pi/180)) ) * ( 1+modulating_index_voltage*math.cos(2*math.pi*modulating_frequency*t_inst + (phase_of_voltage_modulant*(math.pi/180)) ))
    vc_inst = voltage_peak*math.cos( 2*math.pi*base_frequency*t_inst + (phase_of_base_vc*(math.pi/180)) ) * ( 1+modulating_index_voltage*math.cos(2*math.pi*modulating_frequency*t_inst + (phase_of_voltage_modulant*(math.pi/180)) ))
    
    samples_va.append(va_inst)
    samples_vb.append(vb_inst)
    samples_vc.append(vc_inst)
    
    v_alpha = (np.sqrt(2/3)) * ( (1 * va_inst) + ((-1/2) * vb_inst) + ((-1/2) * vc_inst) )
    v_beta  = (np.sqrt(2/3)) * ( (0 * va_inst) + (((np.sqrt(3))/2) * vb_inst) + ((-(np.sqrt(3))/2) * vc_inst) )
    v_gamma = (np.sqrt(2/3)) * ( ((1/(np.sqrt(2))) * va_inst) + ((1/(np.sqrt(2))) * vb_inst + ((1/(np.sqrt(2))) * vc_inst)) )
    
    samples_v_alpha.append(v_alpha)
    samples_v_beta.append(v_beta)
    samples_v_gamma.append(v_gamma)
    
    # Generate values for all currents
    ia_inst = current_peak*math.cos(2*math.pi*base_frequency*t_inst + (phase_of_base_ia*(math.pi/180))) * (1+modulating_index_current*math.cos(2*math.pi*modulating_frequency*t_inst + (phase_of_current_modulant*(math.pi/180)) ))
    ib_inst = current_peak*math.cos(2*math.pi*base_frequency*t_inst + (phase_of_base_ib*(math.pi/180))) * (1+modulating_index_current*math.cos(2*math.pi*modulating_frequency*t_inst + (phase_of_current_modulant*(math.pi/180)) ))
    ic_inst = current_peak*math.cos(2*math.pi*base_frequency*t_inst + (phase_of_base_ic*(math.pi/180))) * (1+modulating_index_current*math.cos(2*math.pi*modulating_frequency*t_inst + (phase_of_current_modulant*(math.pi/180)) ))
    
    
    samples_ia.append(ia_inst)
    samples_ib.append(ib_inst)
    samples_ic.append(ic_inst)

    i_alpha = (np.sqrt(2/3)) * ( (1 * ia_inst) + ((-1/2) * ib_inst) + ((-1/2) * ic_inst) )
    i_beta  = (np.sqrt(2/3)) * ( (0 * ia_inst) + (((np.sqrt(3))/2) * ib_inst) + ((-(np.sqrt(3))/2) * ic_inst) )
    i_gamma = (np.sqrt(2/3)) * ( ((1/(np.sqrt(2))) * ia_inst) + ((1/(np.sqrt(2))) * ib_inst + ((1/(np.sqrt(2))) * ic_inst)) )

    samples_i_alpha.append(i_alpha)
    samples_i_beta.append(i_beta)
    samples_i_gamma.append(i_gamma)
    
    ## compute and append the p_park and q_park values
    
    p_park_inst = (v_alpha * i_alpha) + (v_beta* i_beta)
    q_park_inst = (v_beta * i_alpha) - (v_alpha * i_beta)
    
    samples_P_park.append(p_park_inst)
    samples_Q_park.append(q_park_inst)


## create a dat file which will be located in same root at this python script
## And will be named XXXXX.dat 
Utilities.write_dat_to_file()


## plots to show generated signals

Utilities.make_plot(t[1:600],[samples_va[1:600],samples_vb[1:600],samples_vc[1:600]],"seconds","voltage",["phase A","phase B","phase C"])
Utilities.make_plot(t[1:600],[samples_ia[1:600],samples_ib[1:600],samples_ic[1:600]],"seconds","current",["phase A","phase B","phase C"])
Utilities.make_plot(t[1:600],[samples_v_alpha[1:600],samples_v_beta[1:600],samples_v_gamma[1:600]],"seconds","voltage",["V Alpha","V Beta","V Gamma"])
Utilities.make_plot(t[1:600],[samples_i_alpha[1:600],samples_i_beta[1:600],samples_i_gamma[1:600]],"seconds","voltage",["i Alpha","i Beta","i Gamma"])
Utilities.make_plot(t[1:600],[samples_P_park[1:600],samples_Q_park[1:600]],"seconds","power",["P Park","Q Park"])

## This section of code is to generate rms values for analog signals va,vb,vc,ia,ib and ic
## note that for this cell to work you will need to run the cell 3 steps above this one


def compute_rms_va(samples_va):

    ## define the sampling 

    sampling_range_value = round((ComtradeConfig.sampling_rate()/ComtradeConfig.frequency())/2) 
    sampling_range = np.arange(0,sampling_range_value,1)

    sampling_window = np.arange(0,round(len(samples_va)/sampling_range_value),1)
    rms = []
    time = []
    for i in sampling_window:
    
        voltage_squared = 0
    
        for d in sampling_range:
            voltage_squared += pow(samples_va[(i*sampling_range_value)+d],2)
    
        rms.append(math.sqrt((1/len(sampling_range))*voltage_squared))
        time.append(t[i*sampling_range_value])

    ## plot to show rms signal and va
    plt.figure()
    plt.grid()
    plt.plot(time[1:30],rms[1:30])
    plt.plot(t[1:600],samples_va[1:600])

    ## plot to show rms signal and v_park
    plt.figure()
    plt.grid()
    plt.plot(time[1:30],rms[1:30])
    plt.plot(t[1:600],samples_v_alpha[1:600])

    #scatter plot to show number of samples 
    plt.figure()
    plt.grid()
    plt.scatter(time[1:20],rms[1:20])

    ## figure to show modulated frequency
    plt.figure()
    plt.grid()
    plt.plot(t[1:200],samples_va[1:200])

    max_num = np.max(rms)
    min_num = np.min(rms)
    median = (max_num+min_num)/2
    print("the max rms is = " + str(max_num))
    print("the median rms is = " + str(median))
    print("the min rms is = " + str(min_num))

compute_rms_va()

## datapoints to hold the samples to be fourier transformed

datapoints = []

N = 100

for inst in np.arange(0,N,1):
    datapoints.append(rms[inst])
y = fft(datapoints)
x = fftfreq(N,1/(ComtradeConfig.frequency()*2))

y_scaled = y*(2/N)
print(len(y_scaled))
print(len(x))


plt.figure()
plt.xlabel("frequency")
plt.ylabel("voltage max")
plt.grid(visible=True,which='both')
plt.scatter(x,y_scaled)

sum_of_squares_y = 0
for i in y:
    sum_of_squares_y += pow(np.abs(i),2)
print("sum of squares = " + str(sum_of_squares_y))
rms_total = ((1/N)*math.sqrt(sum_of_squares_y))

print("rms is = " + str(rms_total))

import cmath

angles_for_existing_frequency_bins = []
index = np.arange(0,len(y),1)

for frequency in index:
    num = y_scaled[frequency]
    #print(num)
    c_mag,c_angle = cmath.polar(num)
    angle_degree = c_angle*(180/math.pi)
    
    ## if frequency value is greater than modulating coefficient - note set modulating coefficient to be minimum modulating coefficient for more than one modulating frequency
    if(np.abs(y_scaled[frequency]) > ComtradeConfig.voltage_peak()*modulating_index_voltage*0.9):
        angles_for_existing_frequency_bins.append(angle_degree)
        
    else:
        angles_for_existing_frequency_bins.append(0)

plt.figure()
plt.xlabel("frequency in hertz")
plt.ylabel("angles in degrees")
plt.grid(visible=True,which='both')
plt.scatter(index,angles_for_existing_frequency_bins)


NameError: name 'generate_data_cfg' is not defined